In [1]:
import pandas as pd
import numpy as np
from itertools import chain

from gensim.corpora import Dictionary
import ruptures as rpt

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_json('data/yle-articles.json', orient='records')

In [3]:
df.head()

,id,date,title,body,subjects
0,3-6475540,2013-01-31T22:53:33+0200,UMK-finalistit valittu – Great Wide North maks...,Great Wide North sai toisessa semifinaalissa ...,"[kulttuuri, musiikki, viihde, uuden musiikin k..."
1,3-6475536,2013-01-31T22:53:15+0200,YK:n pääsihteeri: Tieto Israelin iskusta Syyri...,Ban Ki-moon kehottaa Lähi-idän kaikki valtioi...,"[ulkomaat, yhdistyneiden kansakuntien pääsihte..."
2,3-6475539,2013-01-31T22:53:04+0200,AIK kuritti Rögleä - Pirnes kahmi tehopisteitä,Esa Pirnes oli tehokkaalla pelipäällä Elitser...,"[urheilu, jääkiekko, aik if, svenska hockeylig..."
3,20-162900,2013-01-31T22:43:34+0200,Polkupyörällä Islannissa: kirja lähtee painoon,"On taas se aika vuodesta, kun olo on melko ep...","[matkakertomukset, matkakirjallisuus (kaunokir..."
4,3-6475529,2013-01-31T22:38:24+0200,TPS:n oljenkorsi ei katkennut - 36 pistettä yh...,TPS säilytti SaiPasta otetun 3-1-voiton ansio...,"[urheilu, jääkiekon miesten sm-liiga, jääkiekk..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702673 entries, 0 to 702672
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        702673 non-null  object
 1   date      702673 non-null  object
 2   title     702673 non-null  object
 3   body      702673 non-null  object
 4   subjects  702673 non-null  object
dtypes: object(5)
memory usage: 26.8+ MB


In [19]:
# dictionary = Dictionary.load('models/dev/dictionary.pkl')

In [4]:
def create_subjects_dict(subjects_list):
    # subjects = subjects_list.apply(lambda x: list(chain.from_iterable(i.lower().split(' ') for i in x)))
    
    # subjects = list(chain.from_iterable(item.lower().split(' ') for block in subjects_list for item in block))
    # print(subjects[0])
    total = subjects_list.apply(len).sum()
    print(total)
    subject_dict = Dictionary(subjects_list)

    return subject_dict

In [14]:
sub_dict = create_subjects_dict(df['subjects'])

9771037


In [15]:
len(sub_dict)

98392

In [10]:
sub_dict.cfs[2]

1

In [11]:
sub_dict[653]

'sillat'

In [5]:
def count_subjects(sub_dict):

    dict_freq = {}

    for i, word in enumerate(sub_dict):
        dict_freq[sub_dict[i]] = sub_dict.cfs[i]

    count = 0
    for k in sorted(dict_freq, key=dict_freq.get, reverse=True):
        print(k, dict_freq[k])
        count  += 1
        if dict_freq[k] <= 200:
            break

    print(count)

In [4]:
# generate distinct groups for testing the hypothesis
def extract_categories(df, cats):
    clusters = []
    
    for cat in cats:
        is_cat = df['subjects'].apply(lambda x: (cat in x))
        df_filtered = df[is_cat].copy()
        df_filtered['category'] = cat
        clusters.append(df_filtered)
        
    df_merged = pd.concat(clusters, ignore_index=True)
    
    return df_merged

In [5]:
%%time
cat_list = ['autot', 'musiikki', 'luonto', 'vaalit', 'taudit', 'työllisyys', 'jääkiekko', 'kulttuuri', 'rikokset', 'koulut', 'tulipalot', 'ruoat']
df_cluster = extract_categories(df, cats=cat_list)

CPU times: user 3.22 s, sys: 3.93 ms, total: 3.22 s
Wall time: 3.24 s


In [6]:
df_cluster.head()

,id,date,title,body,subjects,category
0,3-6475275,2013-01-31T16:59:42+0200,Aika kallis Volkkari,Vuonna 1973 autonvalmistaja Volkswagen oli kr...,"[tekniikka, ulkomaat, liikenne, vw passat, vol...",autot
1,3-6475075,2013-01-31T16:02:43+0200,Kuopion koruryöstäjille kymmenien tuhansien sa...,Poliisi kaipaa edelleen havaintoja keskiviikk...,"[kotimaan uutiset, ryöstö, varallisuusrikokset...",autot
2,3-6474561,2013-01-31T13:24:08+0200,Nuori nainen kuoli kolarissa Virroilla,Etelä-Pohjanmaalla asuva nuori nainen menehty...,"[virrat, räntä, liikenneonnettomuudet, henkilö...",autot
3,3-6474214,2013-01-31T10:54:11+0200,Autoasentajat kurssikierteessä,Kun auton polttimonvaihto vie alan korjaamoll...,"[kotimaan uutiset, tekniikka, tuulilasinpesin,...",autot
4,3-6474222,2013-01-31T10:54:01+0200,12 kertaa kiinni kortitta ajosta,Kokemäellä autoillut mies jäi keskiviikkona k...,"[ajokortitta ajaminen, kokemäki, liikennerikko...",autot


In [7]:
df_cluster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222779 entries, 0 to 222778
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        222779 non-null  object
 1   date      222779 non-null  object
 2   title     222779 non-null  object
 3   body      222779 non-null  object
 4   subjects  222779 non-null  object
 5   category  222779 non-null  object
dtypes: object(6)
memory usage: 10.2+ MB


In [8]:
df_cluster.to_json('data/dev/cluster_12_cats.json')